In [ ]:
import os, sys, warnings
import numpy as np, matplotlib.pyplot as plt, pandas as pd, seaborn as sns
from sequentia import *
from tqdm.auto import tqdm

# Silence TensorFlow warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Import utility functions and classes
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from utils import *
from lstm import LSTMClassifier

# Filter warnings
warnings.filterwarnings('ignore')

# ggplot style
plt.style.use('ggplot')

# Set seed for reproducible randomness
seed = 0
np.random.seed(seed)
rng = np.random.RandomState(seed=seed)

In [ ]:
gestures_map = {
    'nd': 'nod',
    'mnd': 'multiple nods',
    'fu': 'face-up',
    'fd': 'face-down',
    'sh': 'shake',
    't': 'turn',
    'ti': 'tilt'
}

gestures = list(gestures_map.keys())

In [ ]:
fields = ['Rx', 'Ry', 'Rz', 'Tx', 'Ty', 'Tz']

In [ ]:
# Containers to store classifiers and results
clfs, results = {}, {'hmm': {}, 'knn': {}, 'lstm': {}}

In [ ]:
# Load the MoCap dataset
loader = MoCapLoader(normalized=False)
X, y = loader.load(fields)

## Dataset splits

Create a stratified 65-20-15 training, validation and test set split.

In [ ]:
# Create a stratified training, validation and test set split (65-20-15)
X_train, X_val, X_test, y_train, y_val, y_test = data_split(X, y, random_state=rng, stratify=True)

In [ ]:
# MoCap dataset class counts (training set)
show_class_counts(y_train, gestures, title=None)

In [ ]:
# Histogram of MoCap dataset gesture durations (training set)
show_durations(X_train, bins=75, title=None)

## Preprocessing

In [ ]:
# Create a preprocessing pipeline
pre = Preprocess([
    Filter(window_size=10, method='median'),
    BinDownsample(bin_size=50, method='decimate'),
    MinMaxScale()
])
pre.summary()

In [ ]:
# Function for visualizing gesture signals
def plot_gesture(gesture, label, figsize=(10, 5), same_scale=True, title=None):
    title = "Head rotation and translation vectors for a '{}' gesture".format(gestures_map[label]) if title is None else title
    labels = ['$R_x$ (Roll)', '$R_y$ (Yaw)', '$R_z$ (Pitch)', '$T_x$', '$T_y$', '$T_z$']
    colors = ['blue', 'red', 'green'] * 2
    fig, axs = plt.subplots(3, 2, sharex=True, figsize=figsize)
    
    for i in range(3):
        ax1, ax2 = axs[i, 0], axs[i, 1]
        ax1.plot(gesture[:, i], label=labels[i], color=colors[i])
        ax2.plot(gesture[:, i+3], label=labels[i+3], color=colors[i+3])
        ax1.legend(loc='lower right')
        ax2.legend(loc='lower right')
        if same_scale:
            pad = 0.1
            ax1.set_ylim(gesture.min()-pad, gesture.max()+pad)
            ax2.set_ylim(gesture.min()-pad, gesture.max()+pad)
            
    fig.tight_layout()
    fig.subplots_adjust(top=0.92)
    fig.suptitle(title)
    plt.show()

In [ ]:
# Pick an example signal for visualization
x_sample, y_sample = X_train[0], y_train[0]
plot_gesture(x_sample, y_sample, title="Head rotation vectors for a '{}' gesture (before preprocessing)".format(y_sample))
plot_gesture(pre.transform(x_sample), y_sample, title="Head rotation vectors for a '{}' gesture (after preprocessing)".format(y_sample))

In [ ]:
# Transform training data and plot histogram of MoCap dataset gesture durations (training set)
Xp_train = pre.fit_transform(X_train, verbose=True)
show_durations(Xp_train, bins=75, title=None)

In [ ]:
# Apply the preprocessing pipeline to the other dataset splits
Xp_val, Xp_test = pre.transform(X_val, verbose=True), pre.transform(X_test, verbose=True)

## DTWKNN classifier

### Fitting the model

In [ ]:
%%time
# Create and fit a DTWKNN classifier using the single nearest neighbor and a radius of 1
# NOTE: The radius parameter is a parameter that constrains the FastDTW algorithm.
clfs['knn'] = DTWKNN(k=1, radius=1)
clfs['knn'].fit(Xp_train, y_train)

### Evaluating the model

In [ ]:
%%time
results['knn']['validation'] = clfs['knn'].evaluate(Xp_val, y_val, labels=gestures, n_jobs=-1)
show_results(*results['knn']['validation'], dataset='validation', labels=gestures)

## Hidden Markov Model classifier

One gesture model $\lambda_i=(A_i,B_i,\pi_i)$ is initialized and trained for each of the gestures: `nd`, `mnd`, `sh`, `fd`, `t`, `ti`, `fu`.

### Fitting the model

In [ ]:
%%time

# Create HMMs to represent each class
#
# NumPy sometimes raises some errors as a result of instability during the Cholesky decomposition.
# According to issue #414 on Pomegranate's GitHub repository, this may be caused by:
# - Too many states in the HMMs
# - Too many dimensions in the input data, which leads to a large covariance matrix
# - Too few training examples
hmms = []
for g in tqdm(gestures, desc='Training HMMs'):
    hmm = HMM(label=g, n_states=7, random_state=rng)
    hmm.set_random_initial()
    hmm.set_random_transitions()
    hmm.fit([Xp_train[i] for i, label in enumerate(y_train) if label == g])
    hmms.append(hmm)
    
# Fit a HMM classifier with the HMMs
clfs['hmm'] = HMMClassifier()
clfs['hmm'].fit(hmms)

### Evaluating the model

In [ ]:
%%time
results['hmm']['validation'] = clfs['hmm'].evaluate(Xp_val, y_val)
show_results(*results['hmm']['validation'], dataset='validation', labels=gestures)

In [ ]:
with np.printoptions(precision=3, suppress=True):
    display(hmms[0].initial, hmms[0].transitions)

## LSTM classifier

### Fitting the model

In [ ]:
%%time
from tensorflow.keras.optimizers import Adam
clfs['lstm'] = LSTMClassifier(epochs=125, batch_size=128, optimizer=Adam(learning_rate=0.002), classes=gestures)
hist = clfs['lstm'].fit(Xp_train, y_train, validation_data=(Xp_val, y_val), return_history=True)

In [ ]:
# Summarize the model
clfs['lstm'].summary()

In [ ]:
# Display accuracy and loss history during training
show_accuracy_history(hist)
show_loss_history(hist)

### Evaluating the model

In [ ]:
%%time
results['lstm']['validation'] = clfs['lstm'].evaluate(Xp_val, y_val)
show_results(*results['lstm']['validation'], dataset='validation', labels=gestures)